In [24]:
import pandas as pd
import requests
import json
import sys
from pprint import pprint
from sqlalchemy import create_engine

In [25]:
url_base = "https://app.ticketmaster.com/discovery/v2/events.json?"

sys.path.append('/Users/tokar/UNC_DA/MY_WORK/Personal_Code')
from my_api_config import ticket_master_key

url_country = "&countryCode=US"

url_event = "&classificationName=music"




In [26]:
# Build the query url

query_url = url_base + "apikey=" + ticket_master_key + url_country + url_event

In [34]:
response = requests.get(query_url).json()
dtype(response)

{'_embedded': {'events': [{'name': 'P!NK: Summer Carnival 2024', 'type': 'event', 'id': 'vvG17Z9MBX0JyU', 'test': False, 'url': 'https://www.ticketmaster.com/pnk-summer-carnival-2024-foxborough-massachusetts-08-21-2024/event/01005F7EBE926E76', 'locale': 'en-us', 'images': [{'ratio': '4_3', 'url': 'https://s1.ticketm.net/dam/a/8cc/0850a9c7-f269-4506-87f5-0acb3e2e08cc_CUSTOM.jpg', 'width': 305, 'height': 225, 'fallback': False}, {'ratio': '16_9', 'url': 'https://s1.ticketm.net/dam/a/8cc/0850a9c7-f269-4506-87f5-0acb3e2e08cc_RETINA_PORTRAIT_16_9.jpg', 'width': 640, 'height': 360, 'fallback': False}, {'ratio': '16_9', 'url': 'https://s1.ticketm.net/dam/a/8cc/0850a9c7-f269-4506-87f5-0acb3e2e08cc_SOURCE', 'width': 2426, 'height': 1365, 'fallback': False}, {'ratio': '3_2', 'url': 'https://s1.ticketm.net/dam/a/8cc/0850a9c7-f269-4506-87f5-0acb3e2e08cc_ARTIST_PAGE_3_2.jpg', 'width': 305, 'height': 203, 'fallback': False}, {'ratio': '16_9', 'url': 'https://s1.ticketm.net/dam/a/8cc/0850a9c7-f269-45

In [39]:
response.dtype()

AttributeError: 'dict' object has no attribute 'dtype'

In [36]:
print(json.dumps(response, indent=4, sort_keys=False))

{
    "_embedded": {
        "events": [
            {
                "name": "P!NK: Summer Carnival 2024",
                "type": "event",
                "id": "vvG17Z9MBX0JyU",
                "test": false,
                "url": "https://www.ticketmaster.com/pnk-summer-carnival-2024-foxborough-massachusetts-08-21-2024/event/01005F7EBE926E76",
                "locale": "en-us",
                "images": [
                    {
                        "ratio": "4_3",
                        "url": "https://s1.ticketm.net/dam/a/8cc/0850a9c7-f269-4506-87f5-0acb3e2e08cc_CUSTOM.jpg",
                        "width": 305,
                        "height": 225,
                        "fallback": false
                    },
                    {
                        "ratio": "16_9",
                        "url": "https://s1.ticketm.net/dam/a/8cc/0850a9c7-f269-4506-87f5-0acb3e2e08cc_RETINA_PORTRAIT_16_9.jpg",
                        "width": 640,
                        "height": 36

In [44]:
data = response
events = data["_embedded"]["events"]
filtered_events = [{"name": event["name"], "type": event["type"], "id": event["id"]} for event in events]

# Convert the filtered data to JSON
filtered_json_response = json.dumps(filtered_events, indent=4, sort_keys=False)

print(filtered_json_response)

[
    {
        "name": "P!NK: Summer Carnival 2024",
        "type": "event",
        "id": "vvG17Z9MBX0JyU"
    },
    {
        "name": "P!NK: Summer Carnival 2024",
        "type": "event",
        "id": "vvG1YZ9ASfN7Jv"
    },
    {
        "name": "P!NK: Summer Carnival 2024",
        "type": "event",
        "id": "vvG18Z9Mzzn05q"
    },
    {
        "name": "P!NK: Summer Carnival 2024",
        "type": "event",
        "id": "vvG1FZ9MMoSUKo"
    },
    {
        "name": "P!NK: Summer Carnival 2024",
        "type": "event",
        "id": "vvG1bZ9MwOsvtU"
    },
    {
        "name": "P!NK: Summer Carnival 2024",
        "type": "event",
        "id": "k7vGF9MQsrwdx"
    },
    {
        "name": "P!NK: Summer Carnival 2024",
        "type": "event",
        "id": "k7vGF9z6xPpDI"
    },
    {
        "name": "P!NK Live 2024",
        "type": "event",
        "id": "vvG1HZ90nBI3Rk"
    },
    {
        "name": "P!NK: Summer Carnival 2024",
        "type": "event",
        "id": "